In [ ]:
%pip install selenium
%pip install beautifulsoup4
%pip install plotly
%pip install seaborn
%pip install scipy
%pip install nbformat

In [71]:
%pip freeze | findstr nbformat


nbformat==5.10.4
Note: you may need to restart the kernel to use updated packages.


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

In [35]:
from selenium.webdriver.chrome.service import Service

download_service = Service()

driver = webdriver.Chrome(service=download_service)

driver.maximize_window()

The chromedriver version (131.0.6778.69) detected in PATH at I:\ChromeDriver\chromedriver.exe might not be compatible with the detected chrome version (132.0.6834.110); currently, chromedriver 132.0.6834.110 is recommended for chrome 132.*, so it is advised to delete the driver in PATH and retry


In [36]:
# Otwieranie strony Steam
url = "https://store.steampowered.com/category/science_fiction/"
driver.get(url)

# Czekanie na załadowanie treści
time.sleep(2)  # Dostosuj, jeśli strona ładuje się szybciej

In [62]:
driver.execute_script('window.scroll(0, document.body.scrollHeight-2000)')


In [29]:
load_more_button = driver.find_element(By.CLASS_NAME, '_2tkiJ4VfEdI9kq1agjZyNz.Focusable')
load_more_button.click()

In [37]:
# Znajdź i kliknij przycisk zamykający popup
popup_close_button = driver.find_element(By.ID, "rejectAllButton")
popup_close_button.click()

In [38]:
menu_button = driver.find_element(By.XPATH, "//div[text()='Typ']")
driver.execute_script("arguments[0].scrollIntoView(true);", menu_button)
menu_button.click()


In [39]:
# Znalezienie elementu rodzica (div) dla <a> z tekstem "Gry"
games_button = driver.find_element(By.XPATH, "//a[text()='Gry']")
# Kliknięcie elementu rodzica
driver.execute_script("arguments[0].click();", games_button)


In [40]:
import pandas as pd

def GetTitlesWithDetails():
    games_data = []
    game_containers = driver.find_elements(By.XPATH, "//div[contains(@class, 'v9uRg57bwOaPsvAnkXESO')]")

    for game in game_containers:
        try:
            # Pobranie tytułu gry
            title = game.find_element(By.XPATH, ".//div[contains(@class, '_3rrH9dPdtHVRMzAEw82AId')]//a//div").text.strip()

            # Pobranie tagów
            tags_container = game.find_element(By.XPATH, ".//div[contains(@class, '_3wryhCRrTuMULeq_YjNk-s')]")
            tag_elements = tags_container.find_elements(By.XPATH, ".//div[contains(@class, '_2bkP-3b7dvr0a_qPdZEfHY')]//a")
            tags = ', '.join([tag.text.strip() for tag in tag_elements if tag.text.strip()])

            # Pobranie ceny gry
            try:
                price_container = game.find_element(By.XPATH, ".//div[contains(@class, 'kW6m4Sjqacp5hykrj5LEo')]//div[contains(@class, '_2s-O5T3qJJYR2AUq4b9jIN')]")
                price = price_container.find_element(By.XPATH, ".//div[contains(@class, '_3j4dI1yA7cRfCvK8h406OB')]").text.strip()
            except:
                price = "Free to Play"

            # Pobranie daty wydania
            try:
                release_date = game.find_element(By.XPATH, ".//div[contains(@class, '_3wryhCRrTuMULeq_YjNk-s')]//div[contains(@class, '_3a6HRK-P6LK0-pxRKXYgyP')]//div[contains(@class, '_1qvTFgmehUzbdYM9cw0eS7')]").text.strip()
            except:
                release_date = "Brak daty"

            # Pobranie nacechowania recenzji
            try:
                review_sentiment = game.find_element(By.XPATH, ".//div[contains(@class, '_3wryhCRrTuMULeq_YjNk-s')]//a[contains(@class, '_3qvppfM_u0yn2jrpoUo8RM')]//div[contains(@class, '_3ZWs0kB-1tuqQtie9KK-E7')]//div[contains(@class, '_2nuoOi5kC2aUI12z85PneA')]").text.strip()
            except:
                review_sentiment = "Brak informacji"

            # Pobranie liczby recenzji
            try:
                review_count = game.find_element(By.XPATH, ".//div[contains(@class, '_3wryhCRrTuMULeq_YjNk-s')]//a[contains(@class, '_3qvppfM_u0yn2jrpoUo8RM')]//div[contains(@class, '_3ZWs0kB-1tuqQtie9KK-E7')]//div[contains(@class, '_1wXL_MfRpdKQ3wZiNP5lrH')]").text.strip()
            except:
                review_count = "0"

            # Dodanie gry do listy
            games_data.append({
                "Tytuł": title,
                "Tagi": tags,
                "Cena": price,
                "Data wydania": release_date,
                "Nacechowanie recenzji": review_sentiment,
                "Liczba recenzji": review_count
            })

        except Exception as e:
            print(f"Problem z przetwarzaniem gry: {e}")

    # Konwersja do DataFrame
    df = pd.DataFrame(games_data)
    return df


In [41]:
# zescrolluj do początku listy
#driver.execute_script('window.scroll(0, document.body.scrollHeight-3000)')

# zecrolluj na dół listy
driver.execute_script('window.scroll(0, document.body.scrollHeight-2000)')

# Powtarzanie procesu 10 razy
for i in range(50):
    load_more_button = driver.find_element(By.CLASS_NAME, '_2tkiJ4VfEdI9kq1agjZyNz.Focusable')
    load_more_button.click()
    time.sleep(2)
    driver.execute_script('window.scroll(0, document.body.scrollHeight-2000)')


data = GetTitlesWithDetails()

In [42]:
# Zamknięcie drivera
driver.quit()

In [46]:
import pandas as pd
import numpy as np

def clean_data(df):
    # 1. Konwersja ceny
    def clean_price(price):
        try:
            if "Free to Play" in price:
                return 0.0
            return float(price.replace(' zł', '').replace(',', '.'))
        except:
            return np.nan

    df['Cena'] = df['Cena'].apply(clean_price)

    # 2. Konwersja liczby recenzji
    def clean_reviews(reviews):
        try:
            return int(reviews.replace('Recenzje użytkowników: ', '').replace(' ', ''))
        except:
            return np.nan

    df['Liczba recenzji'] = df['Liczba recenzji'].apply(clean_reviews)

    # 3. Konwersja daty wydania
    polish_months = {
        'STY': 'Jan', 'LUT': 'Feb', 'MAR': 'Mar', 'KWI': 'Apr', 'MAJ': 'May', 'CZE': 'Jun',
        'LIP': 'Jul', 'SIE': 'Aug', 'WRZ': 'Sep', 'PAŹ': 'Oct', 'LIS': 'Nov', 'GRU': 'Dec'
    }

    def clean_date(date):
        if pd.isna(date):
            return np.nan
        for pl, en in polish_months.items():
            date = date.replace(pl, en)
        try:
            return pd.to_datetime(date, format='%d %b %Y')
        except:
            return np.nan

    df['Data wydania'] = df['Data wydania'].apply(clean_date)

    # 4. Rozbicie tagów na kolumny (one-hot encoding)
    def split_tags(tags):
        return tags.split(', ') if isinstance(tags, str) else []

    # Przygotowanie unikalnych tagów
    all_tags = sorted(set(tag for tags in df['Tagi'].apply(split_tags) for tag in tags))
    
    # Tworzenie macierzy tagów (one-hot encoding)
    tags_matrix = pd.DataFrame(
        {
            f'Tag_{tag}': df['Tagi'].apply(lambda tags: 1 if tag in split_tags(tags) else 0)
            for tag in all_tags
        }
    )
    
    # Łączenie oryginalnego DataFrame z macierzą tagów
    df = pd.concat([df.reset_index(drop=True), tags_matrix.reset_index(drop=True)], axis=1)

    # Usuwanie potencjalnych braków danych (przykład)
    df.dropna(subset=['Cena', 'Liczba recenzji', 'Data wydania'], inplace=True)

    return df


In [48]:
display(df)

,Tytuł,Tagi,Cena,Data wydania,Nacechowanie recenzji,Liczba recenzji,"Tag_2,5D",Tag_2D,Tag_3D,Tag_4X,...,Tag_Znaczące wybory,Tag_Zombie,Tag_Zręcznościowe,Tag_Zły protagonista,Tag_Łamigłówki,Tag_Łamigłówki platformowe,Tag_Łupy,Tag_Śledztwo,Tag_Świetna ścieżka dźwiękowa,Tag_Żeńska postać w roli głównej
0,Marvel Rivals,"Free to Play, Wieloosobowe, Strzelanka z bohat...",0.00,2024-12-06,Bardzo pozytywne,181643,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Subnautica,"Survivalowe w otwartym świecie, Survival, Horr...",45.86,2018-01-24,Przytłaczająco pozytywne,270717,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,DOOM Eternal,"FPS, Akcja, Brutalność, Świetna ścieżka dźwięk...",42.25,2020-03-20,Bardzo pozytywne,169836,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,Among Us,"Wieloosobowe, Sieciowa kooperacja, Dedukcyjne ...",10.79,2018-11-16,Bardzo pozytywne,612000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HELLDIVERS™ 2,"Sieciowa kooperacja, PvE, Strzelanka z perspek...",169.00,2024-02-08,W większości pozytywne,706335,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,Chair F*cking Simulator,"Symulator randkowania, Horror psychologiczny, ...",8.99,2020-07-07,Przytłaczająco pozytywne,570,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
608,Aeon Must Die!,"Akcja, Niezależne, Beat 'em up, Bijatyka 2D, M...",10.49,2021-10-14,Mieszane,86,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
609,tomorrow won't come for those without ██████,"Filozoficzne, Klimatyczne, Minimalistyczne, Ła...",8.99,2020-07-04,Bardzo pozytywne,823,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
610,Morbid: The Seven Acolytes,"Souls-like, Akcja, Przygodowe, Inspirowane Lov...",8.99,2020-12-03,W większości pozytywne,812,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [47]:
import pandas as pd

# Dane wejściowe
df = pd.DataFrame(data)

# Przetwarzanie danych
df = clean_data(df)


In [10]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
import plotly.express as px

def analyze_data(df):
    # 1. Tworzenie tabeli krzyżowej tagów
    def create_crosstab(df):
        all_tags = [tag for tags in df['tags'] for tag in tags]  # Zbiór wszystkich tagów
        unique_tags = sorted(set(all_tags))  # Unikalne tagi
        
        # Tworzenie macierzy obecności tagów
        tag_matrix = pd.DataFrame(0, index=unique_tags, columns=unique_tags)
        for tags in df['tags']:
            for tag1 in tags:
                for tag2 in tags:
                    if tag1 != tag2:  # Ignorujemy współwystępowanie z samym sobą
                        tag_matrix.loc[tag1, tag2] += 1
        
        return tag_matrix

    # 2. Test chi-kwadrat na współwystępowanie tagów
    def chi_square_analysis(tag_matrix):
        chi2_results = []
        for tag1 in tag_matrix.index:
            for tag2 in tag_matrix.columns:
                if tag1 != tag2:  # Test tylko dla różnych tagów
                    obs = [[tag_matrix.loc[tag1, tag2], tag_matrix.loc[tag2, tag1]],
                           [sum(tag_matrix.loc[tag1, :]), sum(tag_matrix.loc[:, tag2])]]
                    _, p, _, _ = chi2_contingency(obs)
                    chi2_results.append((tag1, tag2, p))
        
        return pd.DataFrame(chi2_results, columns=['Tag1', 'Tag2', 'P-value']).sort_values('P-value')

    # 3. Analiza wpływu tagów na ceny
    def tag_price_analysis(df):
        tag_price_data = []
        for tag in sorted(set(tag for tags in df['tags'] for tag in tags)):
            filtered = df[df['tags'].apply(lambda tags: tag in tags)]
            avg_price = filtered['price'].mean()
            tag_price_data.append({'Tag': tag, 'Average Price': avg_price})
        
        return pd.DataFrame(tag_price_data).sort_values('Average Price', ascending=False)

    # Tworzenie tabeli krzyżowej tagów
    tag_matrix = create_crosstab(df)
    print("Tabela krzyżowa tagów:\n", tag_matrix)

    # Analiza współwystępowania tagów z testem chi-kwadrat
    #chi2_results = chi_square_analysis(tag_matrix)
    #print("Wyniki testu chi-kwadrat:\n", chi2_results.head(10))

    # Analiza wpływu tagów na ceny
    tag_price_df = tag_price_analysis(df)
    print("Średnia cena dla poszczególnych tagów:\n", tag_price_df)

    # Wizualizacja danych
    fig1 = px.imshow(tag_matrix, text_auto=True, title="Współwystępowanie tagów (Tabela krzyżowa)")
    fig2 = px.bar(tag_price_df, x='Tag', y='Average Price', title="Średnia cena w zależności od tagu", labels={'Tag': 'Tag', 'Average Price': 'Średnia cena'})
    
    fig1.show()
    fig2.show()

    return {
        "tag_matrix": tag_matrix,
        #"chi2_results": chi2_results,
        "tag_price_analysis": tag_price_df
    }


In [ ]:
# Analiza
analyze_data(df)


In [49]:
from collections import Counter
import pandas as pd
import plotly.express as px

def prepare_pie_chart_data(df, top_n=25):
    # Zliczanie wystąpień tagów
    tags = df['Tagi'].apply(lambda x: x.split(', '))
    tag_counts = Counter(tag for tag_list in tags for tag in tag_list)

    # Podział na najpopularniejsze tagi i kategorię "Other"
    top_tags = tag_counts.most_common(top_n)
    other_count = sum(count for tag, count in tag_counts.items() if tag not in dict(top_tags))
    
    # Przygotowanie danych do DataFrame
    pie_data = pd.DataFrame(top_tags, columns=['Tag', 'Count'])
    if other_count > 0:
        other_row = pd.DataFrame([{'Tag': 'Other', 'Count': other_count}])
        pie_data = pd.concat([pie_data, other_row], ignore_index=True)
    
    return pie_data

# Przygotowanie danych do wykresu
pie_chart_data = prepare_pie_chart_data(df)

# Wizualizacja
fig = px.pie(pie_chart_data, names='Tag', values='Count',
             title='Liczba gier per tag (z kategorią "Other")',
             color_discrete_sequence=px.colors.qualitative.Pastel)
fig.show()


In [50]:
def prepare_pie_chart_reviews(df, top_n=20):
    # Zliczanie liczby recenzji na tag
    tags = df['Tagi'].apply(lambda x: x.split(', '))
    tag_reviews = Counter()

    for idx, tag_list in enumerate(tags):
        for tag in tag_list:
            tag_reviews[tag] += df.iloc[idx]['Liczba recenzji']

    # Podział na najpopularniejsze tagi i kategorię "Other"
    top_tags = tag_reviews.most_common(top_n)
    other_count = sum(count for tag, count in tag_reviews.items() if tag not in dict(top_tags))
    
    # Przygotowanie danych do DataFrame
    pie_data = pd.DataFrame(top_tags, columns=['Tag', 'Review Count'])
    if other_count > 0:
        other_row = pd.DataFrame([{'Tag': 'Other', 'Review Count': other_count}])
        pie_data = pd.concat([pie_data, other_row], ignore_index=True)
    
    return pie_data

# Przygotowanie danych do wykresu
pie_chart_reviews_data = prepare_pie_chart_reviews(df)

# Wizualizacja
fig = px.pie(pie_chart_reviews_data, names='Tag', values='Review Count',
             title='Liczba opinii per tag (z kategorią "Other")',
             color_discrete_sequence=px.colors.qualitative.Pastel)
fig.show()


In [51]:
def prepare_avg_reviews_per_tag(df, top_n=20):
    # Zliczanie liczby recenzji na tag i liczby gier
    tags = df['Tagi'].apply(lambda x: x.split(', '))
    tag_reviews = Counter()
    tag_counts = Counter()

    for idx, tag_list in enumerate(tags):
        for tag in tag_list:
            tag_reviews[tag] += df.iloc[idx]['Liczba recenzji']
            tag_counts[tag] += 1

    # Obliczanie średniej liczby recenzji na tag
    avg_reviews = {tag: tag_reviews[tag] / tag_counts[tag] for tag in tag_reviews}

    # Przygotowanie danych: top N tagów i kategoria "Other"
    sorted_avg_reviews = sorted(avg_reviews.items(), key=lambda x: x[1], reverse=True)
    top_tags = sorted_avg_reviews[:top_n]
    other_count = sum(tag_counts[tag] for tag, _ in sorted_avg_reviews[top_n:])

    # Przygotowanie DataFrame
    pie_data = pd.DataFrame(top_tags, columns=['Tag', 'Average Reviews'])
    if other_count > 0:
        other_row = pd.DataFrame([{'Tag': 'Other', 'Average Reviews': other_count}])
        pie_data = pd.concat([pie_data, other_row], ignore_index=True)
    
    return pie_data

# Przygotowanie danych do wykresu
avg_reviews_per_tag_data = prepare_avg_reviews_per_tag(df)

# Wizualizacja
fig = px.pie(avg_reviews_per_tag_data, names='Tag', values='Average Reviews',
             title='Średnia liczba recenzji na tag (z kategorią "Other")',
             color_discrete_sequence=px.colors.qualitative.Pastel)
fig.show()


In [52]:
import plotly.express as px

fig = px.histogram(df, x='Cena', nbins=20, title='Rozkład cen gier')
fig.update_layout(xaxis_title='Cena', yaxis_title='Liczba gier')
fig.show()


In [81]:
from collections import Counter

# Zliczanie wystąpień tagów
tag_counts = Counter()
for tags in df['Tagi']:
    # Przekształcenie stringa w listę tagów, zakładając, że są oddzielone przecinkami
    tag_list = tags.split(', ')
    tag_counts.update(tag_list)

# Przekształcenie wyników w DataFrame
tag_df = pd.DataFrame(tag_counts.items(), columns=['Tag', 'Count']).sort_values('Count', ascending=False)

# Wizualizacja
fig = px.bar(tag_df.head(20), x='Tag', y='Count', title='Najpopularniejsze tagi')
fig.update_layout(yaxis_title='Liczba gier')
fig.show()


In [88]:
# 1. Zależność między ceną a liczbą recenzji

import plotly.express as px

# Ustalamy kolejność nacechowania recenzji od najbardziej pozytywnych do najmniej pozytywnych
category_order = ['Przytłaczająco pozytywne', 'Bardzo pozytywne', 'W większości pozytywne', 'Pozytywne', 'Mieszane', 'W większości negatywne', 'Przytłaczająco negatywne']

# Tworzymy wykres
fig = px.scatter(df, x='Cena', y='Liczba recenzji', color='Nacechowanie recenzji',
                 title='Cena vs. liczba recenzji', labels={'Cena': 'Cena (zł)', 'Liczba recenzji': 'Liczba recenzji'},
                 hover_data=['Tytuł'],
                 category_orders={'Nacechowanie recenzji': category_order},
                 color_continuous_scale='RdYlGn',  # Ustawienie palety kolorów
                 color_discrete_map={'Przytłaczająco pozytywne': 'green', 'Bardzo pozytywne': 'lightgreen', 
                                     'W większości pozytywne': 'yellow', 'Pozytywne': 'yellow', 
                                     'Mieszane': 'orange', 'W większości negatywne': 'red', 'Przytłaczająco negatywne': 'darkred'})

# Wyświetlamy wykres
fig.show()



In [89]:
# 2. Zależność między ceną a nacechowaniem recenzji

# Mapowanie nacechowania na wartości numeryczne
sentiment_map = {
    'Bardzo pozytywne': 3,
    'Pozytywne': 2,
    'Mieszane': 1,
    'Negatywne': 0
}
df['sentiment_score'] = df['Nacechowanie recenzji'].map(sentiment_map)

# Określenie kolejności dla 'Nacechowanie recenzji'
category_order = ['Przytłaczająco pozytywne', 'Bardzo pozytywne', 'W większości pozytywne', 'Pozytywne', 'Mieszane', 'W większości negatywne', 'Przytłaczająco negatywne']

# Scatter plot
fig = px.box(df, x='Nacechowanie recenzji', y='Cena', title='Cena w zależności od nacechowania recenzji',
             labels={'Cena': 'Cena (zł)', 'Nacechowanie recenzji': 'Nacechowanie recenzji'},
             category_orders={'Nacechowanie recenzji': category_order})  # Dodajemy sortowanie

fig.show()



In [56]:
# 3. Rozkład cen w zależności od daty wydania

df['Data wydania'] = pd.to_datetime(df['Data wydania'], errors='coerce')
df['Rok wydania'] = df['Data wydania'].dt.year

# Scatter plot: Cena vs. Rok wydania
fig = px.scatter(df, x='Rok wydania', y='Cena', color='Nacechowanie recenzji',
                 title='Cena w zależności od roku wydania', labels={'Cena': 'Cena (zł)', 'Rok wydania': 'Rok wydania'},
                 hover_data=['Tytuł'])
fig.show()


In [58]:
# 4. Wpływ liczby recenzji na nacechowanie recenzji



In [90]:
# Interaktywny Dashboard

import plotly.subplots as sp

# Histogram cen
fig1 = px.histogram(df, x='Cena', title='Rozkład cen gier', labels={'Cena': 'Cena (zł)'})

# Scatter plot cena vs liczba recenzji
fig2 = px.scatter(df, x='Cena', y='Liczba recenzji', color='Nacechowanie recenzji',
                  title='Cena vs. liczba recenzji', hover_data=['Tytuł'])

# Box plot cena vs nacechowanie recenzji
fig3 = px.box(df, x='Nacechowanie recenzji', y='Cena', title='Cena vs. nacechowanie recenzji')

# Scatter plot cena vs rok wydania
fig4 = px.scatter(df, x='Rok wydania', y='Cena', color='Nacechowanie recenzji',
                  title='Cena vs. rok wydania', hover_data=['Tytuł'])

# Kombinacja wykresów w jednym dashboardzie
dashboard = sp.make_subplots(rows=2, cols=2, subplot_titles=('Rozkład cen', 'Cena vs. liczba recenzji',
                                                             'Cena vs. nacechowanie recenzji', 'Cena vs. rok wydania'))

dashboard.add_traces(fig1.data, rows=1, cols=1)
dashboard.add_traces(fig2.data, rows=1, cols=2)
dashboard.add_traces(fig3.data, rows=2, cols=1)
dashboard.add_traces(fig4.data, rows=2, cols=2)

dashboard.update_layout(height=800, width=1200, title_text='Dashboard analizy gier')
dashboard.show()


In [91]:
def tags_sentiment_analysis(df, top_n=10):
    sentiment_counts = {}
    for sentiment in df['Nacechowanie recenzji'].unique():
        if pd.isna(sentiment):
            continue
        tags = df[df['Nacechowanie recenzji'] == sentiment]['Tagi'].apply(lambda x: x.split(', '))
        tag_counter = Counter(tag for tag_list in tags for tag in tag_list)
        sentiment_counts[sentiment] = dict(tag_counter)

    # Konwersja na DataFrame i ograniczenie liczby tagów
    sentiment_dfs = []
    for sentiment, counts in sentiment_counts.items():
        sentiment_df = pd.DataFrame(counts.items(), columns=['Tag', 'Count'])
        sentiment_df['Sentiment'] = sentiment
        sentiment_dfs.append(sentiment_df)

    combined_df = pd.concat(sentiment_dfs)
    top_tags = combined_df.groupby('Tag')['Count'].sum().nlargest(top_n).index
    return combined_df[combined_df['Tag'].isin(top_tags)]

# Analiza
tags_sentiment_df = tags_sentiment_analysis(df)

# Określenie kolejności dla 'Sentiment'
category_order = ['Przytłaczająco pozytywne', 'Bardzo pozytywne', 'W większości pozytywne', 'Pozytywne', 
                  'Mieszane', 'W większości negatywne', 'Przytłaczająco negatywne']

# Wizualizacja
fig = px.bar(tags_sentiment_df, x='Tag', y='Count', color='Sentiment', barmode='group',
             title='Powiązanie najczęściej występujących tagów z nacechowaniem recenzji',
             labels={'Count': 'Liczba wystąpień', 'Tag': 'Tag'},
             category_orders={'Sentiment': category_order})  # Dodajemy sortowanie dla sentimentu

fig.update_layout(xaxis_tickangle=45)
fig.show()

In [61]:
# Mapowanie tekstowego nacechowania recenzji na wartości numeryczne
sentiment_map = {
    'Przytłaczająco pozytywne': 4,
    'Bardzo pozytywne': 3,
    'W większości pozytywne': 2,
    'Mieszane': 1,
    'Negatywne': 0
}

df['sentiment_score'] = df['Nacechowanie recenzji'].map(sentiment_map).fillna(0)


In [62]:
# Obliczanie średnich ocen ważonych liczbą recenzji oraz średnich cen
def avg_sentiment_and_price_per_tag(df, top_n=20, min_games=3):
    tags = df['Tagi'].apply(lambda x: x.split(', '))
    tag_scores = Counter()
    tag_weights = Counter()
    tag_prices = Counter()
    tag_counts = Counter()

    for idx, tag_list in enumerate(tags):
        for tag in tag_list:
            sentiment_score = df.iloc[idx]['sentiment_score']
            review_count = df.iloc[idx]['Liczba recenzji']
            price = df.iloc[idx]['Cena']
            
            # Ważenie ocen liczbą recenzji
            tag_scores[tag] += sentiment_score * review_count
            tag_weights[tag] += review_count
            
            # Suma cen i liczba wystąpień
            tag_prices[tag] += price
            tag_counts[tag] += 1

    # Obliczanie średnich ocen i cen (filtr minimalnej liczby gier)
    avg_data = {
        tag: {
            'Average Sentiment': tag_scores[tag] / tag_weights[tag],
            'Average Price': tag_prices[tag] / tag_counts[tag],
            'Game Count': tag_counts[tag]
        }
        for tag in tag_scores if tag_counts[tag] >= min_games
    }
    avg_df = pd.DataFrame.from_dict(avg_data, orient='index').reset_index()
    avg_df.rename(columns={'index': 'Tag'}, inplace=True)
    return avg_df.sort_values('Average Sentiment', ascending=False).head(top_n), \
           avg_df.sort_values('Average Sentiment', ascending=True).head(top_n)


In [73]:
import plotly.graph_objects as go

# Obliczanie wyników
top_positive_tags, top_negative_tags = avg_sentiment_and_price_per_tag(df, top_n=15, min_games=3)

def create_dual_axis_chart(df, title):
    fig = go.Figure()

    # Słupki dla średniej oceny
    fig.add_trace(go.Bar(
        x=df['Tag'],
        y=df['Average Sentiment'],
        name='Średnia ocena',
        marker_color='#90CAF9',  # Łagodny pastelowy niebieski
        yaxis='y1'
    ))

    # Linie i punkty dla średniej ceny
    fig.add_trace(go.Scatter(
        x=df['Tag'],
        y=df['Average Price'],
        name='Średnia cena',
        mode='lines+markers',
        marker=dict(color='red', size=10),
        line=dict(width=2, dash='dot'),
        yaxis='y2'
    ))

    # Konfiguracja osi Y
    fig.update_layout(
        title=title,
        xaxis=dict(title='Tagi', tickangle=45),
        yaxis=dict(
            title='Średnia ocena',
            titlefont=dict(color='#90CAF9'),  # Dopasowany kolor tytułu osi
            tickfont=dict(color='#90CAF9'),
        ),
        yaxis2=dict(
            title='Średnia cena (zł)',
            titlefont=dict(color='red'),
            tickfont=dict(color='red'),
            overlaying='y',
            side='right'
        ),
        legend=dict(x=0.5, y=1.1, orientation='h')
    )

    return fig

# Przygotowanie wykresów dla najlepszych i najgorszych tagów
fig_positive = create_dual_axis_chart(top_positive_tags, 'Tagi z najlepszymi ocenami (ważone liczbą recenzji)')
fig_negative = create_dual_axis_chart(top_negative_tags, 'Tagi z najgorszymi ocenami (ważone liczbą recenzji)')

# Wyświetlenie wykresów
fig_positive.show()
fig_negative.show()


In [64]:
def avg_reviews_per_tag(df, top_n=20):
    tags = df['Tagi'].apply(lambda x: x.split(', '))
    tag_reviews = Counter()
    tag_counts = Counter()

    for idx, tag_list in enumerate(tags):
        for tag in tag_list:
            tag_reviews[tag] += df.iloc[idx]['Liczba recenzji']
            tag_counts[tag] += 1

    avg_reviews = {tag: tag_reviews[tag] / tag_counts[tag] for tag in tag_counts}
    avg_reviews_df = pd.DataFrame(list(avg_reviews.items()), columns=['Tag', 'Average Reviews'])
    return avg_reviews_df.nlargest(top_n, 'Average Reviews')

# Analiza
tag_avg_reviews_df = avg_reviews_per_tag(df)

# Wizualizacja
fig = px.bar(tag_avg_reviews_df, x='Tag', y='Average Reviews',
             title='Średnia liczba recenzji na najpopularniejsze tagi',
             labels={'Average Reviews': 'Średnia liczba recenzji'})
fig.update_layout(xaxis_tickangle=45)
fig.show()


In [65]:
def avg_price_per_tag(df, top_n=20):
    tags = df['Tagi'].apply(lambda x: x.split(', '))
    tag_prices = Counter()
    tag_counts = Counter()

    for idx, tag_list in enumerate(tags):
        for tag in tag_list:
            tag_prices[tag] += df.iloc[idx]['Cena']
            tag_counts[tag] += 1

    avg_prices = {tag: tag_prices[tag] / tag_counts[tag] for tag in tag_counts}
    avg_prices_df = pd.DataFrame(list(avg_prices.items()), columns=['Tag', 'Average Price'])
    return avg_prices_df.nlargest(top_n, 'Average Price')

# Analiza
tag_avg_price_df = avg_price_per_tag(df)

# Wizualizacja
fig = px.bar(tag_avg_price_df, x='Tag', y='Average Price',
             title='Średnia cena dla najdroższych tagów',
             labels={'Average Price': 'Średnia cena'})
fig.update_layout(xaxis_tickangle=45)
fig.show()


In [74]:
fig = px.box(df, x='tag_count', y='Cena', title='Rozkład cen w zależności od liczby tagów', labels={'tag_count': 'Liczba tagów', 'Cena': 'Cena (zł)'})
fig.show()



ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['Tytuł', 'Tagi', 'Cena', 'Data wydania', 'Nacechowanie recenzji', 'Liczba recenzji', 'Tag_2,5D', 'Tag_2D', 'Tag_3D', 'Tag_4X', 'Tag_Akcja', 'Tag_Anime', 'Tag_Autobitewniak', 'Tag_Automatyzacja', 'Tag_Battle royale', "Tag_Beat 'em up", 'Tag_Bijatyka', 'Tag_Bijatyka 2D', 'Tag_Bogata fabuła', 'Tag_Bogate uniwersum', 'Tag_Brutalność', 'Tag_Budowanie', 'Tag_Budowanie bazy', 'Tag_Budowanie miasta', 'Tag_Bullet hell', 'Tag_Bullet time', 'Tag_CRPG', 'Tag_Cyberpunk', 'Tag_Czarna komedia', 'Tag_Czarny humor', 'Tag_Czas rzeczywisty', 'Tag_Czas rzeczywisty z pauzą', 'Tag_Dedukcyjne towarzyskie', 'Tag_Demony', 'Tag_Destrukcja', 'Tag_Detektywistyczne', 'Tag_Dinozaury', 'Tag_Dostosowywanie broni palnej', 'Tag_Dostosowywanie postaci', 'Tag_Drużynowe RPG', 'Tag_Dungeon crawler', 'Tag_Dyplomacja', 'Tag_Dystopijne', 'Tag_Dzielony ekran', 'Tag_E-sport', 'Tag_Edukacyjne', 'Tag_Edytor poziomów', 'Tag_Eksperymentalne', 'Ta

In [ ]:
# tabela krzyżowa potrzebna

In [ ]:
tag_prices = []

for tag in all_tags:
    subset = df[df['tags'].apply(lambda tags: tag in tags)]
    avg_price = subset['price'].mean()
    tag_prices.append({'Tag': tag, 'Average Price': avg_price})

tag_price_df = pd.DataFrame(tag_prices).sort_values('Average Price', ascending=False)


# Wybór tagów z największą średnią ceną i różnicą cen
top_expensive_tags = tag_price_df.nlargest(10, 'Average Price')
top_cheap_tags = tag_price_df.nsmallest(10, 'Average Price')
filtered_tag_price_df = pd.concat([top_expensive_tags, top_cheap_tags])

# Wizualizacja
fig = px.bar(filtered_tag_price_df, x='Tag', y='Average Price', title='Wpływ wybranych tagów na cenę', labels={'Tag': 'Tag', 'Average Price': 'Średnia cena'})
fig.show()


In [ ]:
# todo:   - Dla szeregów czasowych, mogą pasować takie metody jak średnie kroczące, metody dekompozycji, itd.

#   - Dla danych kategorycznych, można użyć  jak tabele krzyżowe, test chi-kwadrat,  itd.

#   - Dla danych ciągłych, możemy użyć analizę korelacji, regresję, itd.

In [ ]:
# Modelowanie: Zdecyduj, czy problem to klasyfikacja, czy regresja. Wykorzystaj odpowiednie modele uczenia maszynowego (np. drzewa decyzyjne, regresję logistyczną dla klasyfikacji; regresję liniową, lasy losowe dla regresji) z biblioteki `scikit-learn` i oceń ich wyniki przy użyciu takich miar jak dokładność (accuracy), F1 score lub R-kwadrat (R²), w zależności od typu danych i analizy.

# 1. na podstawie tagów, ceny i roku wydania oceń, jakie opinie ma gra - klasyfikacja
# 2. na podstawie tagów, opinii, liczby opinii, roku wydania zgadnij, jaką cenę ma gra - regresja

In [23]:
%pip install sklearn

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
      rather than 'sklearn' for pip commands.
      
      Here is how to fix this error in the main use cases:
      - use 'pip install scikit-learn' rather than 'pip install sklearn'
      - replace 'sklearn' by 'scikit-learn' in your pip requirements files
        (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
      - if the 'sklearn' package is used by one of your dependencies,
        it would be great if you take some time to track which package uses
        'sklearn' instead of 'scikit-learn' and report it to their issue tracker
      - as a last resort, set the environment variable
        SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
      
      More information is available at
      https://github.com/scikit-learn/sklearn-

In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Przygotowanie danych
tag_matrix = pd.DataFrame(0, index=df.index, columns=all_tags)
for i, tags in enumerate(df['tags']):
    tag_matrix.loc[i, tags] = 1

scaler = StandardScaler()
X = scaler.fit_transform(tag_matrix)

# Klasteryzacja
kmeans = KMeans(n_clusters=5, random_state=42)
df['cluster'] = kmeans.fit_predict(X)

# Wizualizacja klastrów (kolor + centroidy)
fig = px.scatter(df, x='price', y='tag_count', color='cluster', title='Klasteryzacja gier (cena vs liczba tagów)', labels={'price': 'Cena (zł)', 'tag_count': 'Liczba tagów'}, hover_data=['title'])
fig.show()


ModuleNotFoundError: No module named 'sklearn'